In [1]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [2]:
df = pd.read_csv("data.csv", encoding='ISO-8859-1')
df = df[df['CustomerID'].notna()]
df = df[df['Quantity'] > 0]
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='mixed', dayfirst=True, errors='coerce')
df = df.dropna(subset=['InvoiceDate'])
df['CustomerID'] = df['CustomerID'].astype(int)

In [3]:
conn = sqlite3.connect("cltv_project.db")
df.to_sql("transactions", conn, if_exists="replace", index=False)

397924

In [4]:
query = '''
SELECT
    CustomerID,
    MAX(InvoiceDate) AS LastPurchaseDate,
    MIN(InvoiceDate) AS FirstPurchaseDate,
    COUNT(DISTINCT InvoiceNo) AS Frequency,
    SUM(Quantity * UnitPrice) AS Monetary
FROM transactions
GROUP BY CustomerID
'''

rfm = pd.read_sql(query, conn)
rfm['LastPurchaseDate'] = pd.to_datetime(rfm['LastPurchaseDate'])
rfm['FirstPurchaseDate'] = pd.to_datetime(rfm['FirstPurchaseDate'])
latest_date = df['InvoiceDate'].max()
rfm['Recency'] = (latest_date - rfm['LastPurchaseDate']).dt.days
rfm['T'] = (latest_date - rfm['FirstPurchaseDate']).dt.days
rfm = rfm[(rfm['Monetary'] > 0) & (rfm['Frequency'] > 0)]
rfm['AOV'] = rfm['Monetary'] / rfm['Frequency']

In [5]:
X = rfm[['Recency', 'Frequency', 'AOV']]
y = rfm['Monetary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [6]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"MAE: {mae}, RMSE: {rmse}")

MAE: 361.69646858440353, RMSE: 2981.8357886171148


In [7]:
rfm['Predicted_CLTV'] = model.predict(rfm[['Recency', 'Frequency', 'AOV']])

In [8]:
rfm['CLTV_Segment'] = pd.qcut(rfm['Predicted_CLTV'], q=3, labels=['Low', 'Medium', 'High'])

In [9]:
rfm.to_csv("cltv_output.csv", index=False)

In [12]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': [lambda x: (df['InvoiceDate'].max() - x.max()).days,
                    lambda x: x.nunique()],
    'TotalPrice': 'sum'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
summary = rfm.copy()

In [18]:
# Create AOV (Average Order Value) first
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

summary = df.groupby('CustomerID').agg({
    'InvoiceDate': [lambda x: (df['InvoiceDate'].max() - x.max()).days,
                    lambda x: x.nunique()],
    'TotalPrice': 'sum'
}).reset_index()

summary.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
summary['AOV'] = summary['Monetary'] / summary['Frequency']
X = summary[['Recency', 'Frequency', 'AOV']]
summary['predicted_CLTV'] = model.predict(X)
summary['CLTV_Segment'] = pd.qcut(summary['predicted_CLTV'], 3, labels=["Low", "Medium", "High"])
output = summary[['CustomerID', 'predicted_CLTV', 'CLTV_Segment']]
output.to_csv("cltv_predictions.csv", index=False)


In [19]:
import joblib
joblib.dump(model, "cltv_model.pkl")


['cltv_model.pkl']